# Identifying mutation networks using SAMBAR
Author: 
Genís Calderer<sup>1</sup>

<sup>1</sup>[Kuijjer Lab](https://www.kuijjerlab.org/), NCMM, Oslo, Norway. - genis.calderer@gmail.com

# Introduction

SAMBAR<sup>1</sup> is a method to identify subtypes of samples based on annotated mutation data. It uses gene mutation counts and membership of those genes in a set of pathways to summarize the mutation rate in a "pathway mutation score" thus despasifying the mutation data.
For each sample and each pathway a score is computed and it can be used to compare the samples to identify subtypes. The SAMBAR package uses binomial distance and hierarchical clustering to find subgroups of samples with similar pathway mutation patterns.

SAMBAR, or Subtyping Agglomerated Mutations By Annotation Relations, is a method to identify subtypes based on somatic mutation data. SAMBAR was used to identify mutational subtypes in 23 cancer types from The Cancer Genome Atlas<sup>2</sup>. 

SAMBAR's input is a matrix that includes the number of non-synonymous mutations in a sample $i$
and gene $j$. SAMBAR first subsets these data to a set of 2,219 cancer-associated genes (optional) from the Catalogue Of Somatic Mutations In Cancer (COSMIC)<sup>3</sup> and Östlund *et al*<sup>4</sup>, or from a user-defined list. It then divides the number of non-synonymous mutations by the gene's length $L_j$, defined as the number of non-overlapping exonic base pairs of a gene. For each sample, SAMBAR then calculates the overall cancer-associated mutation rate by summing mutation scores in all cancer-associated genes $j'$. It removes samples for which the mutation rate is zero and divides the mutation scores the remaining samples by the sample's mutation rate, resulting in a matrix of mutation rate-adjusted scores $G$:

\begin{equation*}
G_{ij}=\frac{N_{ij}/L_{j}}{\displaystyle\sum_{j'}({N_{ij'}/L_{j'}})}.
\end{equation*}

The next step in SAMBAR is de-sparsification of these gene mutation scores (agglomerated mutations) into pathway mutation (annotation relation) scores. SAMBAR converts a (user-defined) gene signature (.gmt format) into a binary matrix $M$, with information of whether a gene $j$ belongs to a pathway $q$. It then calculates pathway mutation scores $P$ by correcting the sum of mutation scores of all genes in a pathway for the number of pathways $q'$ a gene belongs to, and for the number of cancer-associated genes present in that pathway:

\begin{equation*}
P_{iq}=\frac{\displaystyle\sum_{j \in q} G_{ij}/{\displaystyle\sum_{q'} M_{jq'}}}{\displaystyle\sum_{j} M_{jq}}
\end{equation*}

Finally, SAMBAR uses binomial distance to cluster the pathway mutation scores. The cluster dendrogram is then divided into $k$ groups (or a range of $k$ groups), and the cluster assignments are returned in a list.

This guide will use the toy data included in the netZooPy package to showcase the usage of this function.

# 1. Importing SAMBAR from netZooPy

This tutorial can be ran on the server or locally by specifying the `runserver` parameter.

In [ ]:
runserver=1

In order to use the SAMBAR functions it has to be imported from the netZooPy as follows:

In [ ]:
from netZooPy import sambar
import os
import pandas as pd # To read dataframes

First, we need to set server specific parameters and paths.

In [ ]:
if runserver==1:
    os.chdir('../data')
    ppath=='/opt/data/'    

To see the parameters of the main function one can use the following line:

In [ ]:
help(sambar.sambar)

# 2. Selecting input files

The program requires a gene mutation dataset, a list of gene sizes, a list of pathways with its genes, and a list of cancer-associated genes (optional) . By default, SAMBAR runns on the following example data files:

In [ ]:
mut_file   = ppath+"mut.ucec.csv"
esize_file = ppath+"esizef.csv"
sign_file  = ppath+"h.all.v6.1.symbols.gmt"
cangenes   = ppath+"genes.txt"

## Gene mutation file
The file processing functions are already included in SAMBAR, however, to explore the structure of these files, we can read them again.

In [ ]:
df1 = pd.read_csv(mut_file, index_col=0)
df1

This dataframe contains the observed mutation data for each gene (rows) and each individual sample (columns). The matrix contains the number of observed mutations for each sample. `mut_file` has to be formatted as `csv` file.

## Gene sizes

In [ ]:
df2 = pd.read_csv(esize_file)
df2

This input contains a list of genes and their length by the number of bases formatted as a `csv` file.

## Pathway list

In [ ]:
df3 = pd.read_csv(sign_file, sep='\t', header=None)
df3

Pathway files are standard `gmt` files that contain pathways and the genes associated to them.

## Cancer genes list

In [ ]:
df4 = pd.read_csv(cangenes, sep='\t', header=None)
df4

This optional input file contains a list of cancer genes, with each gene entered in a new line.

# 3. Run SAMBAR

The main SAMBAR function takes as input the filepaths of the datasets and returns a pathway score dataframe and a sample clustering dataframe for different cuts in the linkage tree.
It also outputs a csv file for the adjusted mutation scores, pathway scores and clustering.
The slow step in this method is the computation of the distance matrix, this matrix is also exported in case it's needed and rerunning the whole process is not wanted.
The function runs first the desparcification and then the clustering.

In [ ]:
pathway_scores, cluster_groups = sambar.sambar(mut_file,esize_file,cangenes,sign_file) 

`sambar` can be called with additional parameters such as `kmin` and `kmax` to set the range of clusters we're interested in finding. By default `kmin=2` and `kmax=4`, therefore, the number of clusters should be in the that range.

Calling SAMBAR without any input file, will ran the analysis on the toy data set that we analyzed in the previous step and that is integrated within the netZooPy package.

In [ ]:
pathway_scores, cluster_groups = sambar.sambar() 

# 4. Results

The pathway mutation scores and the sample groups are the output of the method. 

In [ ]:
pathway_scores.head(10)

The pathway score matrix is a pathway-by-sample matrix that contains scores for each sample in each pathway. Scores represent the likelihood of the pathway being mutated in a specific sample.

In [ ]:
cluster_groups.head()

The `cluster_groups` function clusters samples by their mutation scores. Here, The output provides three different clustering solutions corresponding to diffrent cuts in the dendrogram. For the first solution, we see that sample `TCGA-A5-A0G3` is assigned to cluster 0 and `TCGA-A5-A0GA` is assigned in cluster 1. We see that the largest cluster assignement is 3, which correponds to the range specified by `kmin` and `kmax`.

# References

1- Kuijjer, Marieke Lydia, et al. "Cancer subtype identification using somatic mutation data." British journal of cancer 118.11 (2018): 1492-1501.

2- Tomczak, Katarzyna, Patrycja Czerwińska, and Maciej Wiznerowicz. "The Cancer Genome Atlas (TCGA): an immeasurable source of knowledge." Contemporary oncology 19.1A (2015): A68.

3- Tate, John G., et al. "COSMIC: the catalogue of somatic mutations in cancer." Nucleic acids research 47.D1 (2019): D941-D947.

4- Östlund, Gabriel, Mats Lindskog, and Erik LL Sonnhammer. "Network-based Identification of novel cancer genes." Molecular & Cellular Proteomics 9.4 (2010): 648-655.